<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/wikidata_sparql_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sparqlwrapper

# https://rdflib.github.io/sparqlwrapper/

     |████████████████████████████████| 348kB 11.6MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


In [0]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON


def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
      item = []
      for c in cols:
        item.append(row.get(c, {}).get('value'))
      out.append(item)
    return pd.DataFrame(out, columns = cols)

In [0]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?noodle ?noodleLabel ?country_of_origin ?country_of_originLabel ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel  ?Commons_category 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "   [AUTO_LANGUAGE],en". }
  ?noodle wdt:P279 wd:Q192874.
  OPTIONAL { ?noodle wdt:P495 ?country_of_origin. }
  OPTIONAL { ?noodle wdt:P31 ?instance_of. }
  OPTIONAL { ?noodle wdt:P279 ?subclass_of. }
  OPTIONAL { ?noodle wdt:P373 ?Commons_category. }  
}"""


df = get_results(endpoint_url, query)

In [4]:
df.head()

,noodle,noodleLabel,country_of_origin,country_of_originLabel,instance_of,instance_ofLabel,subclass_of,subclass_ofLabel,Commons_category
0,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
1,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q1142,Alsace,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
2,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
3,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q1142,Alsace,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
4,http://www.wikidata.org/entity/Q34156,Korean noodles,http://www.wikidata.org/entity/Q18097,Korea,None,None,http://www.wikidata.org/entity/Q192874,noodle,Noodles from Korea


In [5]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel','Commons_category']]

,noodleLabel,country_of_originLabel,instance_ofLabel,subclass_ofLabel,Commons_category
0,Spätzle,France,None,noodle,Spaetzle
1,Spätzle,Alsace,None,noodle,Spaetzle
2,Spätzle,France,None,egg pasta,Spaetzle
3,Spätzle,Alsace,None,egg pasta,Spaetzle
4,Korean noodles,Korea,None,noodle,Noodles from Korea
...,...,...,...,...,...
76,extruded noodle,None,None,noodle,Extruded noodles
77,knife-cut noodle,None,None,noodle,Knife-cut noodles
78,Mì,Vietnam,None,noodle,None
79,ribbon noodle,None,type of pasta,pasta,None


In [6]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel']]['noodleLabel'].unique()

array(['Spätzle', 'Korean noodles', 'zhajiangmian', 'ramen',
       'Curry noodle', 'udon', 'sōmen', 'soba', 'hot dry noodles',
       'cart noodle', 'cellophane noodles', 'jajangmyeon', 'Bakmi',
       'rice noodles', 'Hokkien mee', 'fried noodles', 'Pancit',
       'Chinese noodles', 'Wonton noodles', 'buckwheat noodles', 'kesme',
       'black noodles', 'Bánh hỏi', 'Char kway teow',
       'Chinkiang pot cover noodles', 'Mi rebus', 'dragon beard noodles',
       'phat si io', 'Vietnamese noodles', 'kishimen', 'Kadaif noodles',
       'cold noodle', 'Buldak Bokkeum Myun', 'Mie ayam',
       'Japanese noodles', 'boat noodles', 'Q24933621',
       'Singapore chow mein', 'Q27566431', 'narrow lapsha', 'wide lapsha',
       'laghman', 'extruded noodle', 'knife-cut noodle', 'Mì',
       'ribbon noodle'], dtype=object)

In [7]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel','Commons_category']]['Commons_category'].unique()

array(['Spaetzle', 'Noodles from Korea', 'Zha jiang mian', 'Ramen', None,
       'Udon', 'Somen', 'Soba', 'Re gan mian', 'Cart noodle',
       'Cellophane noodles', 'Jajangmyeon', 'Noodles from Indonesia',
       'Rice noodle', 'Hokkien mee', 'Fried noodles', 'Pancit',
       'Noodles from China', 'Wonton noodle', 'Kesme', 'Bánh hỏi',
       'Char kway teow', 'Bakmi Jawa', 'Phat si-io',
       'Noodles from Vietnam', 'Kishimen', 'Tel kadayıf', 'Cold noodles',
       'Mie ayam', 'Noodles from Japan', 'Laghman', 'Extruded noodles',
       'Knife-cut noodles'], dtype=object)

In [0]:
df.groupby(['Commons_category','noodleLabel'])['noodleLabel'].agg('count')

Commons_category        noodleLabel       
Bakmi Jawa              Mi rebus              1
Bánh hỏi                Bánh hỏi              1
Cart noodle             cart noodle           2
Cellophane noodles      cellophane noodles    2
Char kway teow          Char kway teow        1
Cold noodles            cold noodle           1
Extruded noodles        extruded noodle       1
Fried noodles           fried noodles         2
Hokkien mee             Hokkien mee           2
Jajangmyeon             jajangmyeon           3
Kesme                   kesme                 2
Kishimen                kishimen              2
Knife-cut noodles       knife-cut noodle      1
Laghman                 laghman               2
Mie ayam                Mie ayam              3
Noodles from China      Chinese noodles       1
Noodles from Indonesia  Bakmi                 2
Noodles from Japan      Japanese noodles      1
Noodles from Korea      Korean noodles        1
Noodles from Vietnam    Vietnamese noodles   

In [0]:
endpoint_url = "https://query.wikidata.org/sparql"
query2 = """SELECT ?subclass ?subclassLabel ?TasteAtlas_ID ?instance_of ?instance_ofLabel ?country_of_origin ?country_of_originLabel ?topic_s_main_template ?topic_s_main_templateLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?subclass wdt:P279 wd:Q41415.
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  OPTIONAL { ?subclass wdt:P495 ?country_of_origin. }
  OPTIONAL { ?subclass wdt:P1424 ?topic_s_main_template. }
}"""




In [0]:
df2 = get_results(endpoint_url, query2)

In [8]:
df2.head()

,subclass,subclassLabel,TasteAtlas_ID,instance_of,instance_ofLabel,country_of_origin,country_of_originLabel,topic_s_main_template,topic_s_main_templateLabel
0,http://www.wikidata.org/entity/Q5183356,Cream Stew,None,http://www.wikidata.org/entity/Q4167410,Wikimedia disambiguation page,http://www.wikidata.org/entity/Q17,Japan,None,None
1,http://www.wikidata.org/entity/Q5188689,crossing the bridge noodles,over-bridge-rice-noodles,None,None,None,None,None,None
2,http://www.wikidata.org/entity/Q5191461,Crème Ninon,None,None,None,None,None,None,None
3,http://www.wikidata.org/entity/Q5194921,Fabes con almejas,None,None,None,None,None,None,None
4,http://www.wikidata.org/entity/Q5202905,Q5202905,None,None,None,None,None,None,None


In [0]:
soup_list = df2.groupby(['subclassLabel'])['country_of_originLabel'].agg('count').index.values.tolist()

In [20]:
len(soup_list)

402

In [22]:
soup_list[0:50]

['15 bean soup',
 '2007 Vietnam food scare',
 'Acquacotta',
 'Aguadito de pollo',
 'Ajoblanco',
 'Alicot',
 'Amiedi',
 'Amish preaching soup',
 'Arabaşı soup',
 'Ash Reshteh',
 'Ashe doogh',
 'Asian soup',
 'Aush',
 'Bacon soup',
 'Bagnun',
 'Banga',
 'Batchoy',
 'Beyran',
 'Binignit',
 'Black sesame soup',
 'Bob chorba',
 'Bogeo',
 'Bookbinder Soup',
 'Borş de burechiuşe',
 'Bouillon',
 'Bouillon cube',
 'Bouneschlupp',
 'Brown Windsor soup',
 'Buddha Jumps Over the Wall',
 'Bun rieu',
 'Buridda',
 'Bún mắm',
 'Bún ốc',
 'Caldillo de perro',
 'Caldo de costilla',
 'Caldo de pollo',
 'Caldo de siete mares',
 'Caldo gallego',
 'Caldo tlalpeño',
 'Caldo verde',
 'Canja de Goa',
 'Cantonese seafood soup',
 'Caparrones',
 'Cazuela',
 'Celimpungan',
 'Chakna',
 'Changua',
 'Chankonabe',
 'Chicken and dumplings',
 'Chikhirtma']

In [23]:
soup_list

['15 bean soup',
 '2007 Vietnam food scare',
 'Acquacotta',
 'Aguadito de pollo',
 'Ajoblanco',
 'Alicot',
 'Amiedi',
 'Amish preaching soup',
 'Arabaşı soup',
 'Ash Reshteh',
 'Ashe doogh',
 'Asian soup',
 'Aush',
 'Bacon soup',
 'Bagnun',
 'Banga',
 'Batchoy',
 'Beyran',
 'Binignit',
 'Black sesame soup',
 'Bob chorba',
 'Bogeo',
 'Bookbinder Soup',
 'Borş de burechiuşe',
 'Bouillon',
 'Bouillon cube',
 'Bouneschlupp',
 'Brown Windsor soup',
 'Buddha Jumps Over the Wall',
 'Bun rieu',
 'Buridda',
 'Bún mắm',
 'Bún ốc',
 'Caldillo de perro',
 'Caldo de costilla',
 'Caldo de pollo',
 'Caldo de siete mares',
 'Caldo gallego',
 'Caldo tlalpeño',
 'Caldo verde',
 'Canja de Goa',
 'Cantonese seafood soup',
 'Caparrones',
 'Cazuela',
 'Celimpungan',
 'Chakna',
 'Changua',
 'Chankonabe',
 'Chicken and dumplings',
 'Chikhirtma',
 'Ching bo leung',
 'Chorba',
 'Chupe',
 'Chè bà ba',
 'Ciorbă',
 'Ciorbă de perișoare',
 'Clam soup',
 'Cocido lebaniego',
 'Cocido madrileño',
 'Cockchafer soup',
 

In [24]:
endpoint_url = "https://query.wikidata.org/sparql"
query3 = """SELECT ?is_a_list_of ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel ?country_of_origin ?country_of_originLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?is_a_list_of wdt:P279 wd:Q21976260.
  OPTIONAL {  }
  OPTIONAL { ?is_a_list_of wdt:P5456 ?instance_of. }
  OPTIONAL { ?is_a_list_of wdt:P279 ?subclass_of. }
  OPTIONAL { ?is_a_list_of wdt:P495 ?country_of_origin. }
}
"""

df3 = get_results(endpoint_url,query3)
df3.head()

,is_a_list_of,instance_of,instance_ofLabel,subclass_of,subclass_ofLabel,country_of_origin,country_of_originLabel
0,http://www.wikidata.org/entity/Q2841189,None,None,http://www.wikidata.org/entity/Q21976260,rice dish,http://www.wikidata.org/entity/Q668,India
1,http://www.wikidata.org/entity/Q2916401,None,None,http://www.wikidata.org/entity/Q21976260,rice dish,None,None
2,http://www.wikidata.org/entity/Q2921726,None,None,http://www.wikidata.org/entity/Q21976260,rice dish,None,None
3,http://www.wikidata.org/entity/Q3239751,None,None,http://www.wikidata.org/entity/Q21976260,rice dish,None,None
4,http://www.wikidata.org/entity/Q3239751,None,None,http://www.wikidata.org/entity/Q27994917,chicken dish,None,None


In [27]:
df3['instance_ofLabel']

0                 None
1                 None
2                 None
3                 None
4                 None
            ...       
207          mujaddara
208    lotus-leaf-rice
209          kamameshi
210             pongal
211             pongal
Name: instance_ofLabel, Length: 212, dtype: object

In [33]:
df3.groupby(['subclass_ofLabel', 'instance_ofLabel'], as_index=False)['country_of_originLabel'].agg('count').instance_ofLabel.values.tolist()

['nasi-campur',
 'biryani',
 'hayashi-rice',
 'sindhi-biryani',
 'arroz-con-pollo',
 'biryani',
 'hokkien-fried-rice',
 'botamochi',
 'kheer',
 'arroz-con-leche',
 'arroz-doce',
 'sutlijas',
 'arroz-con-coco',
 'biryani',
 'chukadon',
 'donburi',
 'gyudon',
 'hokkien-fried-rice',
 'oyakodon',
 'tekkadon',
 'chukadon',
 'gyudon',
 'oyakodon',
 'tekkadon',
 'unadon',
 'loco-moco',
 'tamago-kake-gohan',
 'risotto',
 'biryani',
 'akki-rotti',
 'biryani',
 'pongal',
 'zosui',
 'arroz-caldo',
 'loco-moco',
 'mansaf',
 'arroz-con-leche',
 'arroz-doce',
 'sutlijas',
 'hoedeopbap',
 'sushi',
 'akki-rotti',
 'arroz-caldo',
 'arroz-carreteiro',
 'arroz-con-coco',
 'arroz-con-leche',
 'arroz-con-pollo',
 'arroz-doce',
 'bibimbap',
 'biryani',
 'botamochi',
 'cabidela',
 'calas',
 'chazuke',
 'chukadon',
 'com-lam',
 'daifuku',
 'dal-bhat',
 'donburi',
 'gyudon',
 'hayashi-rice',
 'hoedeopbap',
 'hokkien-fried-rice',
 'idli',
 'kabsa',
 'kamameshi',
 'kayu',
 'ketupat',
 'kheer',
 'kiribath',
 'lem